In [9]:
import bcolz
from sklearn.model_selection import train_test_split
import numpy as np

In [17]:
inputs = bcolz.open('/data/poker/predict_action_input.dat')[30000:]
outputs = bcolz.open('/data/poker/predict_action_output.dat')[30000:]

cards_data = np.array(inputs[:,0].tolist())
bets_data = np.array(inputs[:,1].tolist()).reshape((inputs.shape[0], 3, 5,1))

In [18]:
from keras.layers import Input, Dense, Flatten, concatenate, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv2D, ZeroPadding2D
import keras.layers
from keras.optimizers import Nadam

cards_input = Input(shape=(4, 13, 4))
bets_input = Input(shape=(3,5,1))

cx = ZeroPadding2D(padding=(0, 2))(cards_input)
cx = Conv2D(30,(4,5), padding='same', activation='relu')(cx)
cx = Conv2D(20,(4,5), padding='valid', activation='relu')(cx)
cx = Flatten()(cx)

bx = Conv2D(30,(3,1), padding='valid', activation='relu')(bets_input)
bx = Conv2D(20,(1,1), padding='valid', activation='relu')(bx)
bx = Flatten()(bx)

tx = concatenate([cx, bx])
tx = Dense(100, activation='relu')(tx)
tx = Dropout(0.5)(tx)
tx = Dense(50, activation='relu')(tx)
tx = Dropout(0.5)(tx)
bet_output = Dense(4, name='bet_output', activation='softmax')(tx)

model = Model([cards_input, bets_input], [bet_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.006)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit([cards_data, bets_data], outputs, shuffle=True, validation_split=0.05, epochs=10)

/opt/conda/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 38321 samples, validate on 2017 samples
Epoch 1/10
38321/38321 [==============================] - 5s - loss: 0.8941 - acc: 0.6727 - val_loss: 0.6276 - val_acc: 0.7804
Epoch 2/10
38321/38321 [==============================] - 5s - loss: 0.6794 - acc: 0.7395 - val_loss: 0.5277 - val_acc: 0.8091
Epoch 3/10
38321/38321 [==============================] - 5s - loss: 0.6179 - acc: 0.7654 - val_loss: 0.4983 - val_acc: 0.8240
Epoch 4/10
38321/38321 [==============================] - 5s - loss: 0.5824 - acc: 0.7789 - val_loss: 0.4998 - val_acc: 0.8176
Epoch 5/10
38321/38321 [==============================] - 5s - loss: 0.5615 - acc: 0.7868 - val_loss: 0.4864 - val_acc: 0.8250
Epoch 6/10
38321/38321 [==============================] - 5s - loss: 0.5402 - acc: 0.7950 - val_loss: 0.4571 - val_acc: 0.8255
Epoch 7/10
38321/38321 [==============================] - 5s - loss: 0.5237 - acc: 0.8045 - val_loss: 0.4384 - val_acc: 0.8394
Epoch 8/10
38321/38321 [==============================] - 5s -

In [20]:
model.save_weights('/data/trained_models/poker/predict_action_v2.h5')